# Ingredient Substitution Using UMF

One of the most practical uses of UMF is substituting materials while maintaining glaze chemistry. If you can't get a specific feldspar, or want to replace an oxide with a raw source, UMF helps you find equivalent replacements.

The approach:
1. Calculate the UMF of your original recipe
2. Remove the ingredient you want to replace
3. Add replacement material(s) that provide the same oxides
4. Adjust amounts to match the original UMF

In [ ]:
from utils import read_materials, format_umf_table

In [ ]:
materials = read_materials("""
materials:
  custer_feldspar:
    name: Custer Feldspar
    loi: 0.3
    analysis:
      SiO2: 68.5
      Al2O3: 18.2
      K2O: 10.0
      Na2O: 3.0

  minspar:
    name: Minspar 200
    loi: 0.2
    analysis:
      SiO2: 68.8
      Al2O3: 18.5
      K2O: 4.8
      Na2O: 7.0
      CaO: 0.5

  silica:
    name: Silica
    analysis:
      SiO2: 100.0

  whiting:
    name: Whiting
    loi: 44.0
    analysis:
      CaO: 56.0

  epk:
    name: EPK Kaolin
    loi: 14.0
    analysis:
      SiO2: 46.0
      Al2O3: 38.0
      Fe2O3: 1.0
      TiO2: 1.0

  potash_ite:
    name: PotashIte
    analysis:
      K2O: 100.0

  soda_ash:
    name: Soda Ash
    loi: 41.5
    analysis:
      Na2O: 58.5
""")

In [ ]:
molecular_weight = {
    "SiO2": 60.085, "Al2O3": 101.961, "TiO2": 79.866,
    "Li2O": 29.881, "Na2O": 61.979, "K2O": 94.196,
    "MgO": 40.304, "CaO": 56.077, "SrO": 103.619, "BaO": 153.326,
    "ZnO": 81.379, "Fe2O3": 159.688,
}

flux_oxides = ["Li2O", "Na2O", "K2O", "CaO", "MgO", "SrO", "BaO", "ZnO"]

In [ ]:
def recipe_to_umf(recipe, materials):
    """Calculate UMF from a recipe."""
    total_oxides = {}
    for mat_id, parts in recipe.items():
        mat = materials[mat_id]
        for oxide, pct in mat['analysis'].items():
            total_oxides[oxide] = total_oxides.get(oxide, 0) + parts * pct / 100
    
    fired = sum(total_oxides.values())
    oxide_pct = {ox: val / fired * 100 for ox, val in total_oxides.items()}
    
    moles = {ox: pct / molecular_weight[ox] for ox, pct in oxide_pct.items()}
    flux_total = sum(moles[ox] for ox in moles if ox in flux_oxides)
    
    return {ox: mol / flux_total for ox, mol in moles.items()}

## Original Recipe

A simple cone 10 glaze using Custer Feldspar:

In [ ]:
original = {
    'custer_feldspar': 40,
    'silica': 30,
    'whiting': 20,
    'epk': 10,
}

original_umf = recipe_to_umf(original, materials)
print(format_umf_table(original_umf, flux_oxides=[ox for ox in flux_oxides if ox in original_umf]))

## The Problem

Say Custer Feldspar is unavailable. We have Minspar 200 instead.

Let's compare their analyses:

In [ ]:
print("Oxide      Custer    Minspar   Difference")
print("-" * 45)
custer = materials['custer_feldspar']['analysis']
minspar = materials['minspar']['analysis']
all_oxides = set(custer.keys()) | set(minspar.keys())
for ox in sorted(all_oxides):
    c = custer.get(ox, 0)
    m = minspar.get(ox, 0)
    print(f"{ox:8}   {c:6.1f}    {m:6.1f}    {m-c:+6.1f}")

The main difference: Minspar has more Na₂O and less K₂O. A straight substitution will change the flux balance.

In [ ]:
naive_sub = {
    'minspar': 40,  # direct replacement
    'silica': 30,
    'whiting': 20,
    'epk': 10,
}

naive_umf = recipe_to_umf(naive_sub, materials)

print("Original vs Naive Substitution:")
print(f"{'Oxide':8} {'Original':>10} {'Naive':>10} {'Diff':>10}")
print("-" * 40)
for ox in original_umf:
    o = original_umf[ox]
    n = naive_umf.get(ox, 0)
    print(f"{ox:8} {o:10.3f} {n:10.3f} {n-o:+10.3f}")

## Adjusting the Recipe

To match the original UMF, we need to compensate for Minspar's higher Na₂O and lower K₂O. Options:

1. Accept the difference (often small enough not to matter)
2. Add a K₂O source to compensate
3. Adjust other materials

For educational purposes, let's try option 2 - adding a small amount of a potassium source:

In [ ]:
# Trial and error to match flux balance
adjusted = {
    'minspar': 38,
    'silica': 30,
    'whiting': 20,
    'epk': 10,
    'potash_ite': 2,  # small addition to boost K2O
}

adjusted_umf = recipe_to_umf(adjusted, materials)

print("Original vs Adjusted:")
print(f"{'Oxide':8} {'Original':>10} {'Adjusted':>10} {'Diff':>10}")
print("-" * 40)
for ox in set(original_umf.keys()) | set(adjusted_umf.keys()):
    o = original_umf.get(ox, 0)
    a = adjusted_umf.get(ox, 0)
    print(f"{ox:8} {o:10.3f} {a:10.3f} {a-o:+10.3f}")

## Practical Notes

- **Small differences often don't matter.** A 0.02 change in K₂O:Na₂O ratio rarely affects the fired result noticeably.

- **Focus on what matters for your glaze.** If it's a clear glaze, SiO₂:Al₂O₃ ratio matters most. If color is critical, flux balance matters more.

- **Test!** UMF helps you make educated guesses, but the kiln has the final say.